<a id='home_cell'></a>
## BLOG POST - ANOVA 

*Please note I have chosen to use GitHub / Jupyter notebooks in substitute of Tumblr for my blog assignments

The null hypothesis is that between defaulted and non-defaulted the loans, numerous variables would be the same
between the two categories.  Some of those variables include: interest rate, debt-to-income, and revolving utilization.
    
For this exercise, when exploring my initial variables, I kept receving p-values barely above 0.  < 0.001.  Becuase of 
this, there is strong evidence against the null hypothesis, and I would reject the null hypothesis.  We wouldn't expect
data as extreme as the data we actually observed. 

In [1]:
# imports
import pandas
import numpy
import seaborn
import matplotlib.pyplot as plt

# here the stats
import statsmodels.formula.api as smf
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as multi 

In [2]:
# =====================
# 2. DATA IMPORT 
# =====================
# set working directory
directory = '//Project/OneDrive/Coursera/Wesleyan_Course1/wk1_Assignment/'

pdLoans = pandas.read_csv(directory + 'LoanStats3a.csv', low_memory=False)

# =======================
#  3. DATA TRANSFORMATIONS 
#  ======================= 
# Convert the data elements - just doing in 
pdLoans["int_rate_d"] = pandas.to_numeric(pdLoans["int_rate"].str.replace('%',''), errors='coerce')
pdLoans["revol_util_d"] = pandas.to_numeric(pdLoans["revol_util"].str.replace('%',''), errors='coerce')

# divide by 100 to get the true percentage
pdLoans["int_rate_d"] = pdLoans["int_rate_d"] / 100.00
pdLoans["revol_util_d"] = pdLoans["revol_util_d"] / 100.00

# Convert and Bin the data
pdLoans['int_rate_bin'] = pandas.cut(pdLoans.int_rate_d, [0.0,0.10,0.15,0.20,0.25,0.30])
pdLoans['dti_bin'] = pandas.cut(pdLoans.dti, [0,1,10,20,30])
pdLoans['revol_util_bin'] = pandas.cut(pdLoans.revol_util_d, [0,0.25,0.5,0.75,1])

# Add a veriable for recoveries / collection_recovery_fee.  
# If either is greather than 0, then assign a 1 to indicate a default
# Create a function to Map
def set_Defaulted (row):
    if row['recoveries'] > 0:
        return 1
    if row['collection_recovery_fee'] > 0:
        return 1
    else:
        return 0 

pdLoans['defaulted'] = pdLoans.apply(lambda row: set_Defaulted(row), axis=1)

# Here split out separate date set for the defaulted loans - for analayis use later
pdLoansDefaulted = pdLoans[pdLoans['defaulted'] == 1]

## p-value analysis 
<a id='p_values'></a>

In [3]:
## I believe for this excercise - I can use the defaulted variable against the %'s above

In [4]:
model_int_rate_d = smf.ols(formula='int_rate_d ~ C(defaulted)', data=pdLoans)
results1 = model_int_rate_d.fit()
print(results1.summary())

                            OLS Regression Results                            
Dep. Variable:             int_rate_d   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     1753.
Date:                Mon, 18 Sep 2017   Prob (F-statistic):               0.00
Time:                        16:06:26   Log-Likelihood:                 80645.
No. Observations:               42535   AIC:                        -1.613e+05
Df Residuals:                   42533   BIC:                        -1.613e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.1186      0.00

In [5]:
model_revol_util_d = smf.ols(formula='revol_util_d ~ C(defaulted)', data=pdLoans)
results2 = model_revol_util_d.fit()
print(results2.summary())

                            OLS Regression Results                            
Dep. Variable:           revol_util_d   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     384.3
Date:                Mon, 18 Sep 2017   Prob (F-statistic):           3.51e-85
Time:                        16:06:27   Log-Likelihood:                -6552.7
No. Observations:               42445   AIC:                         1.311e+04
Df Residuals:                   42443   BIC:                         1.313e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.4799      0.00

In [6]:
model_dti = smf.ols(formula='dti ~ C(defaulted)', data=pdLoans)
results3 = model_dti.fit()
print(results3.summary())

                            OLS Regression Results                            
Dep. Variable:                    dti   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     72.31
Date:                Mon, 18 Sep 2017   Prob (F-statistic):           1.90e-17
Time:                        16:06:27   Log-Likelihood:            -1.4139e+05
No. Observations:               42535   AIC:                         2.828e+05
Df Residuals:                   42533   BIC:                         2.828e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            13.2570      0.03

In [7]:
# After p-values above being 0, I want to take a closer look at the just the variables to be analyzed 
print('mean of defaulted loans')
pdSubSetVariables = pdLoans[['int_rate_d','dti','revol_util_d','defaulted','loan_amnt']]
mean1 = pdSubSetVariables.groupby('defaulted').mean()
print(mean1)

mean of defaulted loans
           int_rate_d        dti  revol_util_d     loan_amnt
defaulted                                                   
0            0.118560  13.256994      0.479929  10961.596386
1            0.139267  14.034706      0.555376  11820.245785


In [8]:
# The defaulte value (0 or 1 for defaulted loan or not) didn't really produce anything.
# So I changed it up to show the Pupose of the loan instead of defaulted value
model_dti_purpose = smf.ols(formula='int_rate_d ~ C(purpose)', data=pdLoans)
results4 = model_dti_purpose.fit()
print(results4.summary())

                            OLS Regression Results                            
Dep. Variable:             int_rate_d   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     87.78
Date:                Mon, 18 Sep 2017   Prob (F-statistic):          1.26e-232
Time:                        16:06:28   Log-Likelihood:                 80349.
No. Observations:               42535   AIC:                        -1.607e+05
Df Residuals:                   42521   BIC:                        -1.605e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

## post-hoc test

In [9]:
# Post Hoc test with Tukey
mc1 = multi.MultiComparison(pdLoans['int_rate_d'], pdLoans['purpose'])
res1 = mc1.tukeyhsd()
print(res1.summary())

          Multiple Comparison of Means - Tukey HSD,FWER=0.05         
      group1             group2       meandiff  lower   upper  reject
---------------------------------------------------------------------
       car            credit_card      0.0119   0.0084  0.0154  True 
       car         debt_consolidation  0.0195   0.0164  0.0227  True 
       car            educational      0.0147   0.008   0.0214  True 
       car          home_improvement   0.0094   0.0057  0.0132  True 
       car               house         0.0176   0.0109  0.0243  True 
       car           major_purchase    0.0036  -0.0004  0.0076 False 
       car              medical        0.0103   0.0049  0.0157  True 
       car               moving        0.011    0.0052  0.0168  True 
       car               other         0.0133   0.0098  0.0169  True 
       car          renewable_energy   0.0088  -0.0035  0.0211 False 
       car           small_business    0.0251   0.021   0.0292  True 
       car          